In [1]:
import speech_recognition as sr
import pyttsx3
import webbrowser
import requests
from keras.models import load_model
import cv2
import numpy as np
import pyttsx3
import smtplib 


Using TensorFlow backend.


In [2]:
def detect_mask(lock):                                  #mask algorithm using model-018 for both fintie and infinite frames
    model = load_model('model-018.model')

    face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

    source=cv2.VideoCapture(0)

    labels_dict={0:'MASK',1:'NO MASK'}
    color_dict={0:(0,255,0),1:(0,0,255)}
    loop_count=0
    label=2
    while(True):

        ret,img=source.read()
        gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        faces=face_clsfr.detectMultiScale(gray,1.3,5)  

        for x,y,w,h in faces:
    
            face_img=gray[y:y+w,x:x+w]
            resized=cv2.resize(face_img,(100,100))
            normalized=resized/255.0
            reshaped=np.reshape(normalized,(1,100,100,1))
            result=model.predict(reshaped)

            label=np.argmax(result,axis=1)[0]
      
            cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
            cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
            cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
            
        
        cv2.imshow('LIVE',img)
        key=cv2.waitKey(1)
        loop_count=loop_count+1
        if(lock==1 and loop_count>=50):     # for check mask quick with 50 frames
            eng = pyttsx3.init()
            if(label==1):
                eng.say("People with no mask detected")
                eng.runAndWait()
                break
            if(label==0):
                eng.say("People with mask detected")
                eng.runAndWait()
                break
            if(label==2):
                eng.say("No person detected ")
                eng.runAndWait()
                break
            
                
                
        
        if(lock==0 and label==1):
            eng = pyttsx3.init()
            eng.say("People with no mask detected")
            eng.runAndWait()
            label=2
            if(key==27):
                break
            
    
        if(key==27):
            break
        
    cv2.destroyAllWindows()
    source.release()

In [3]:
def danger_master():
    ip_request = requests.get('https://get.geojs.io/v1/ip.json')
    my_ip = ip_request.json()['ip']
    geo_request = requests.get('https://get.geojs.io/v1/ip/geo/' +my_ip + '.json')
    geo_data = geo_request.json()
    print({'latitude': geo_data['latitude'], 'longitude': geo_data['longitude']})
    
        
    #s = smtplib.SMTP('smtp.gmail.com', 587) 
    #s.starttls() 

    # Authentication 
    #s.login("xyz@gmail.com", "Sender password") 

    # message to be sent 
    #message = "This is a emergency mail regarding here is the coordinates too his/her location{'latitude': geo_data['latitude'], 'longitude': geo_data['longitude']}"

    # sending the mail 
    #s.sendmail("xyz@gmail.com", "abc@gmail.com", message) 

    # terminating the session 
    #s.quit()

In [4]:
r0=sr.Recognizer()
r1=sr.Recognizer()
r2=sr.Recognizer()
r3=sr.Recognizer()

In [5]:
with sr.Microphone() as Source:
    engine = pyttsx3.init()
    engine.say("Hello! I am grumm. At your service master.")
    
    engine.runAndWait()
    
    r3.adjust_for_ambient_noise(Source,duration=0)
    audio=r3.listen(Source)
    
    try:
        if 'check mask' in r2.recognize_google(audio):
            eng1 = pyttsx3.init()
            eng1.say("Mask Check Activated.")
            eng1.runAndWait()
                     # Algorithm of mask with 20 frames    
            detect_mask(lock=1)                #lock=1 for making the while loop break after 50 frames
        if 'mask mode' in r1.recognize_google(audio):
            eng2 = pyttsx3.init()
            eng2.say("Activating continious survailance mode.")
            eng2.runAndWait()
            # Algorithm of mask with infinite frames 
            detect_mask(lock=0)                     #lock=0
        
        
        if 'danger' in r2.recognize_google(audio):
        #Algorithm to get the longitude and lattitude coordinates and mail it to emergency contacts andnearest police station 
           danger_master()
           engine = pyttsx3.init()
           engine.say("Mail sent sucessfully.")
           engine.runAndWait()
        
        
        
    except:
        eng3 = pyttsx3.init()
        eng3.say("Try again master.")
        

        
        
        
        